In [ ]:
import os
import json
import PIL.Image
import imgaug.augmenters as iaa
import tensorflow as tf
import mrcnn.utils
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

# Dataset directory
dataset_dir = "data/processed"

# Train directory
train_dir = os.path.join(dataset_dir, "train")

# Validation directory
validation_dir = os.path.join(dataset_dir, "validation")

# Test directory
test_dir = os.path.join(dataset_dir, "test")

# Create a directory to store the processed dataset
os.makedirs(dataset_dir, exist_ok=True)

# Resize the images and convert polygon annotations to masks
annotations_path = "path/to/annotations.json"
images_dir = "/path/to/images"

# Load annotations from JSON file
with open(annotations_path, "r") as file:
    annotations = json.load(file)

for annotation in annotations:
    image_filename = annotation["filename"]
    image_path = os.path.join(images_dir, image_filename)
    image = PIL.Image.open(image_path)
    image = image.resize((224, 224))
    image.save(os.path.join(dataset_dir, image_filename))

    # Convert polygon annotation to mask
    polygons = annotation["polygons"]
    mask = mrcnn.utils.polygons_to_mask(image.size[::-1], polygons)
    mask_path = os.path.join(dataset_dir, image_filename.replace(".jpg", ".png"))
    mask_image = PIL.Image.fromarray(mask.astype("uint8") * 255)
    mask_image.save(mask_path)

# Augment the data (optional)
augmenter = iaa.Fliplr(0.5)
for image in os.listdir(train_dir):
    if image.endswith(".jpg"):
        image_path = os.path.join(train_dir, image)
        augmented_image = augmenter.augment_image(PIL.Image.open(image_path))
        augmented_image.save(image_path)

# Create the model configuration
class SunglassesConfig(Config):
    NAME = "sunglasses"
    IMAGES_PER_GPU = 2
    NUM_CLASSES = 2  # Background + Sunglasses
    STEPS_PER_EPOCH = 100
    DETECTION_MIN_CONFIDENCE = 0.9

config = SunglassesConfig()

# Create the model
model = MaskRCNN(mode="training", config=config, model_dir=dataset_dir)

# Train the model
model.train(
    train_dir,
    validation_dir,
    epochs=100,
    batch_size=8,
    learning_rate=0.0001,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=10, factor=0.1),
        tf.keras.callbacks.EarlyStopping(patience=20),
    ],
)

# Evaluate the model
model.evaluate(validation_dir)

# Save the model
model.save(os.path.join(dataset_dir, "model.h5"))

In [ ]:
https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb